# Images
> Images

- toc: true 
- badges: true
- comments: true

# Grayscale

In [2]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String grayscaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.grayscaleFile = this.outDir + name + "Gray." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void grayscale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.grayscaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int red = (int)(col.getRed() * 0.299);
                    int green = (int)(col.getGreen() * 0.587);
                    int blue = (int)(col.getBlue() * 0.114); 
                    Color rgb = new Color(red + green + blue, red + green + blue, red + green + blue);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.grayscaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.grayscale(); 

        
    }
}
Pics.main(null);

![](images/tmp/MonaLisaGray.png)

# Red

In [5]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String redscaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.redscaleFile = this.outDir + name + "Red." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void redscale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.redscaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int red = (int)(col.getRed());
                    Color rgb = new Color(red, 0, 0);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.redscaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.redscale(); 

        
    }
}
Pics.main(null);

![](images/tmp/MonaLisaRed.png)

# Blue

In [6]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String bluescaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.bluescaleFile = this.outDir + name + "Blue." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void bluescale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.bluescaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int blue = (int)(col.getBlue());
                    Color rgb = new Color(0, 0, blue);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.bluescaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.bluescale(); 

        
    }
}
Pics.main(null);

![](images/tmp/MonaLisaBlue.png)

# Green

In [4]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String greenscaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.greenscaleFile = this.outDir + name + "Green." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void greenscale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.greenscaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int green = (int)(col.getGreen());
                    Color rgb = new Color(0, green, 0);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.greenscaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.greenscale(); 

        
    }
}
Pics.main(null);

![](images/tmp/MonaLisaGreen.png)